Imported all necessary libraries 

In [28]:
import pandas as pd
import matplotlib
import glob
import uuid
import numpy as np
import sqlalchemy

First, data for newark airport weather 2016 was read into a pandas dataframe. Then, only selected necessary columns were selected and information on non-null values was checked.

In [29]:
weather_file = '/Users/simarbhatia/Documents/Codecademy_Projects/bike-rental-starter-kit/data/newark_airport_2016.csv'
weather_df = pd.read_csv(weather_file)
selected_cols_indices = [0,1,2,3,5,6,7,8,9,10,12,13,14,15]
weather_df = weather_df.iloc[:,selected_cols_indices]
weather_df.columns = [x.replace(' ','_').lower() for x in weather_df.columns]
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   station  366 non-null    object 
 1   name     366 non-null    object 
 2   date     366 non-null    object 
 3   awnd     366 non-null    float64
 4   prcp     366 non-null    float64
 5   snow     366 non-null    float64
 6   snwd     366 non-null    float64
 7   tavg     366 non-null    int64  
 8   tmax     366 non-null    int64  
 9   tmin     366 non-null    int64  
 10  wdf2     366 non-null    int64  
 11  wdf5     364 non-null    float64
 12  wsf2     366 non-null    float64
 13  wsf5     364 non-null    float64
dtypes: float64(7), int64(4), object(3)
memory usage: 40.2+ KB


Displayed number of unique values for each column in weather_df dataframe.

In [30]:
weather_df.nunique()

station      1
name         1
date       366
awnd        70
prcp        56
snow        11
snwd        12
tavg        69
tmax        74
tmin        69
wdf2        34
wdf5        35
wsf2        32
wsf5        44
dtype: int64

Previewed the first 5 rows of the weather_df dataframe.

In [31]:
weather_df.head()

,station,name,date,awnd,prcp,snow,snwd,tavg,tmax,tmin,wdf2,wdf5,wsf2,wsf5
0,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-01,12.75,0.0,0.0,0.0,41,43,34,270,280.0,25.9,35.1
1,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-02,9.40,0.0,0.0,0.0,36,42,30,260,260.0,21.0,25.1
2,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-03,10.29,0.0,0.0,0.0,37,47,28,270,250.0,23.9,30.0
3,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-04,17.22,0.0,0.0,0.0,32,35,14,330,330.0,25.9,33.1
4,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-05,9.84,0.0,0.0,0.0,19,31,10,360,350.0,25.1,31.1


The purpose of the code below is to create columns that allow better analysis of bike rental and weather data plus improve readability. 
* The name column is split into airport, state and country.
    * Airport name is reformated so that only the first letter of each word is capitalized
    * State is kept to the 2 letter abbreviation.
    * Country is kept to the 2 letter abbreviation.
* A datekey column was created to create a unique id for each day's weather data in 2016 and also used to link this information to bike rental information.
* Rainfall column was created to indicate if there was any rain according to prcp column value.
* Snowfall column was created to indicate if there was any snow according to snow column value.

In [32]:

weather_df[['airport','state-country']] = weather_df['name'].str.split(',',expand = True)
weather_df['state-country'] = weather_df['state-country'].str.strip()
weather_df[['state','country']] = weather_df['state-country'].str.split(r'\s+',n=1,expand=True)
weather_df['airport'] = weather_df['airport'].str.title()
weather_df['datekey'] = weather_df['date'].astype(str).str.replace('-', '', regex=False)
weather_df['date'] = pd.to_datetime(weather_df['date'])
weather_df['rain_fall'] = weather_df['prcp'].apply(lambda x: 1 if x > 0 else 0).astype(bool)
weather_df['snow_fall'] = weather_df['snow'].apply(lambda x: 1 if x > 0 else 0).astype(bool)
weather_df = weather_df[['station','airport','state','country','date','datekey','awnd','prcp','snow','rain_fall','snow_fall','snwd','tavg','tmax','tmin','wdf2','wdf5','wsf2','wsf5']]
weather_df.head()

,station,airport,state,country,date,datekey,awnd,prcp,snow,rain_fall,snow_fall,snwd,tavg,tmax,tmin,wdf2,wdf5,wsf2,wsf5
0,USW00014734,Newark Liberty International Airport,NJ,US,2016-01-01,20160101,12.75,0.0,0.0,False,False,0.0,41,43,34,270,280.0,25.9,35.1
1,USW00014734,Newark Liberty International Airport,NJ,US,2016-01-02,20160102,9.40,0.0,0.0,False,False,0.0,36,42,30,260,260.0,21.0,25.1
2,USW00014734,Newark Liberty International Airport,NJ,US,2016-01-03,20160103,10.29,0.0,0.0,False,False,0.0,37,47,28,270,250.0,23.9,30.0
3,USW00014734,Newark Liberty International Airport,NJ,US,2016-01-04,20160104,17.22,0.0,0.0,False,False,0.0,32,35,14,330,330.0,25.9,33.1
4,USW00014734,Newark Liberty International Airport,NJ,US,2016-01-05,20160105,9.84,0.0,0.0,False,False,0.0,19,31,10,360,350.0,25.1,31.1


The df_date dataframe was created using the weather_df to provide information on parts of the date: full date, month, day, month_name, day_name, weekend.

In [33]:
df_date = weather_df[['datekey','date']]
df_date['month']= df_date['date'].apply(lambda x: x.month)
df_date['day']= df_date['date'].apply(lambda x: x.day)
df_date['month_name']= df_date['date'].apply(lambda x: x.strftime('%B'))
df_date['day_name'] = df_date['date'].apply(lambda x: x.strftime('%A'))
df_date['weekend'] = df_date['day_name'].apply(lambda x: 1 if (x == 'Saturday' or x=='Sunday') else 0).astype(bool)


#df_date['month_name'] = df_date['full_date'].apply(lambda x: x.strftime('%B'))
#df_date['day_name'] = df_date['full_date'].apply(lambda x: x.strftime('%A'))
df_date.head()

/var/folders/sr/mm1y0d8x5j7_402ymp7jz4dm0000gn/T/ipykernel_39412/471367710.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_date['month']= df_date['date'].apply(lambda x: x.month)
/var/folders/sr/mm1y0d8x5j7_402ymp7jz4dm0000gn/T/ipykernel_39412/471367710.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_date['day']= df_date['date'].apply(lambda x: x.day)
/var/folders/sr/mm1y0d8x5j7_402ymp7jz4dm0000gn/T/ipykernel_39412/471367710.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy 

,datekey,date,month,day,month_name,day_name,weekend
0,20160101,2016-01-01,1,1,January,Friday,False
1,20160102,2016-01-02,1,2,January,Saturday,True
2,20160103,2016-01-03,1,3,January,Sunday,True
3,20160104,2016-01-04,1,4,January,Monday,False
4,20160105,2016-01-05,1,5,January,Tuesday,False


Added quarter information column to df_date. 

In [34]:
def get_quarter(x):    
    if x <= 3:
        return 1
    elif x <= 6:
        return 2
    elif x <= 9:
        return 3
    else:
        return 4        

# add financial quarters
df_date['financial_qtr'] = df_date['month'].apply(get_quarter)
df_date.head()

/var/folders/sr/mm1y0d8x5j7_402ymp7jz4dm0000gn/T/ipykernel_39412/3981965542.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_date['financial_qtr'] = df_date['month'].apply(get_quarter)


,datekey,date,month,day,month_name,day_name,weekend,financial_qtr
0,20160101,2016-01-01,1,1,January,Friday,False,1
1,20160102,2016-01-02,1,2,January,Saturday,True,1
2,20160103,2016-01-03,1,3,January,Sunday,True,1
3,20160104,2016-01-04,1,4,January,Monday,False,1
4,20160105,2016-01-05,1,5,January,Tuesday,False,1


All duplicates for weather station and date information were removed.

In [35]:
weather_station = weather_df[['station','airport','state','country']]
weather_station.rename(columns={'station': 'id'}, inplace=True)
weather_station = weather_station.drop_duplicates()

weather_df.rename(columns={'station':'station_id'}, inplace = True)
date_info = df_date
date_info = date_info.drop_duplicates()
weather_info = weather_df[['station_id','datekey','awnd','prcp','snow','rain_fall', 'snow_fall','snwd','tavg','tmax','tmin','wdf2','wdf5','wsf2','wsf5']]
weather_info = weather_info.drop_duplicates()
weather_info.head()

/var/folders/sr/mm1y0d8x5j7_402ymp7jz4dm0000gn/T/ipykernel_39412/4181686288.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_station.rename(columns={'station': 'id'}, inplace=True)


,station_id,datekey,awnd,prcp,snow,rain_fall,snow_fall,snwd,tavg,tmax,tmin,wdf2,wdf5,wsf2,wsf5
0,USW00014734,20160101,12.75,0.0,0.0,False,False,0.0,41,43,34,270,280.0,25.9,35.1
1,USW00014734,20160102,9.40,0.0,0.0,False,False,0.0,36,42,30,260,260.0,21.0,25.1
2,USW00014734,20160103,10.29,0.0,0.0,False,False,0.0,37,47,28,270,250.0,23.9,30.0
3,USW00014734,20160104,17.22,0.0,0.0,False,False,0.0,32,35,14,330,330.0,25.9,33.1
4,USW00014734,20160105,9.84,0.0,0.0,False,False,0.0,19,31,10,360,350.0,25.1,31.1


Created a connection between dataframes and postgresql tables to export data to designated tables in the databases. 

In [36]:
#db_connection_str = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{Portfolio Projects}'
user = 'postgres'
password = 'simkb415'
host = 'localhost'
port = 5432
database = 'citi_bikes_nyc'
engine = sqlalchemy.create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

con = engine.connect()

date_info.to_sql(name = 'date_info', con=engine,schema = 'citi_bike_weather', if_exists='replace',index=False,chunksize=10000)
weather_station.to_sql(name = 'weather_station', con=engine,schema = 'citi_bike_weather', if_exists='replace',index=False,chunksize=10000)
weather_info.to_sql(name = 'weather_info',con=engine,schema = 'citi_bike_weather', if_exists='replace',index=False,chunksize=10000)

366